<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStarstuff/blob/main/Fast_Gradient_Sign_Method_(FGSM)_for_Adversarial_Example_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

# Define and load the model (example: ResNet18)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
model.fc = nn.Linear(model.fc.in_features, 10)  # Assuming CIFAR-10 with 10 classes
model = model.to(device)

# Load the test dataset
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))  # CIFAR-10 normalization
])

test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define the FGSM attack function
def fgsm_attack(model, images, labels, epsilon):
    images.requires_grad = True
    outputs = model(images)
    loss = F.cross_entropy(outputs, labels)
    model.zero_grad()
    loss.backward()
    perturbed_image = images + epsilon * images.grad.sign()
    return torch.clamp(perturbed_image, 0, 1)

# Function to calculate accuracy
def calculate_accuracy(model, loader):
    correct = 0
    total = 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    return 100 * correct / total

# Evaluate on clean images
clean_accuracy = calculate_accuracy(model, test_loader)
print(f'Accuracy on clean images: {clean_accuracy:.2f}%')

# Evaluate on adversarial examples
epsilon = 0.1  # Perturbation magnitude
correct = 0
total = 0

model.eval()  # Set model to evaluation mode
for images, labels in test_loader:
    images, labels = images.to(device), labels.to(device)

    # Generate adversarial examples
    perturbed_images = fgsm_attack(model, images, labels, epsilon)

    # Re-classify the perturbed images
    outputs = model(perturbed_images)
    _, predicted = torch.max(outputs.data, 1)

    total += labels.size(0)
    correct += (predicted == labels).sum().item()

# Calculate accuracy
adversarial_accuracy = 100 * correct / total
print(f'Accuracy on adversarial examples: {adversarial_accuracy:.2f}%')